# Hidden Markov Model on group elements

## Construct the Cayley graph

In [44]:
import networkx as nx
import numpy as np

# Construct a Cayley graph of the group PSL(2, 7)

A = np.array([[1, 1], [0, 1]])
B = np.array([[0, 1], [-1, 0]])
A_inv = np.array([[1, -1], [0, 1]])  # Inverse of A in PSL(2,7)
generator_set = {
    'A' : A,
    'A_inv' : A_inv,
    'B' : B
}
d = 10  # Diameter for BFS
G = nx.DiGraph()

# Helper function to reduce matrix modulo 7
def mod7(matrix):
    return matrix % 7

# Normalize matrix to canonical projective representative
# In PSL(2, 7), we identify M with -M (i.e., M with 6M mod 7)
def normalize_projective(matrix):
    m = mod7(matrix)
    m_neg = mod7(-m)  # This is 6*m mod 7
    
    # Choose the lexicographically smaller one
    m_flat = m.flatten()
    m_neg_flat = m_neg.flatten()
    
    for i in range(4):
        if m_flat[i] < m_neg_flat[i]:
            return m
        elif m_flat[i] > m_neg_flat[i]:
            return m_neg
    return m

# Helper function to find node with given element
def find_node_with_elem(elem_normalized):
    for node, attrs in G.nodes(data=True):
        if np.array_equal(attrs['elem'], elem_normalized):
            return node
    return None

ind = 0
identity = np.eye(2, dtype=int)
G.add_node(ind, elem=normalize_projective(identity), word_len=0, path=[])

prev_count = 0
for path_length in range(d):
    # Get nodes at current path length
    nodes_at_length = [node for node, attrs in G.nodes(data=True) if attrs['word_len'] == path_length]
    
    for v in nodes_at_length:
        v_elem = G.nodes[v]['elem']
        
        for a, label in [(A, 'A'), (A_inv, 'A_inv'), (B, 'B')]:
            new_elem = normalize_projective(mod7(v_elem @ a))
            
            # Check if this element already exists as a node
            existing_node = find_node_with_elem(new_elem)
            
            if existing_node is None:
                # Add new node
                ind += 1
                G.add_node(ind, elem=new_elem, word_len=path_length + 1, path=G.nodes[v]['path'] + [label])
                G.add_edge(v, ind, generator=label)
            else:
                # Add edge to existing node
                G.add_edge(v, existing_node, generator=label)
    
    # Check if we're still finding new nodes
    current_count = G.number_of_nodes()
    if current_count == prev_count:
        print(f"No new nodes found at depth {path_length}. Stopping early.")
        break
    prev_count = current_count

print(f"\nTotal nodes: {G.number_of_nodes()}")
print(f"Total edges: {G.number_of_edges()}")
print(f"Expected: 168 nodes for PSL(2, 7)")


Total nodes: 168
Total edges: 498
Expected: 168 nodes for PSL(2, 7)


In [48]:
G.nodes[1]['path']

['A']

### The transmission matrix

In [57]:
# Define the state transition matrix
import itertools
from tqdm import tqdm
adjacent_nodes = {i+1 : [] for i in range(2)}
adjacent_elems = {i+1 : [] for i in range(2)}
prob_weights = [0.8/3, 0.2/6]
for node, attrs in G.nodes(data=True):
    if attrs['word_len'] <= 2 and attrs['word_len'] > 0:
        adjacent_nodes[attrs['word_len']].append(node)
        adjacent_elems[attrs['word_len']].append(attrs['elem'])


def inverse_path(path):
    inv_path = []
    for label in path:
        if label == 'A':
            inv_path.append('A_inv')
        elif label == 'A_inv':
            inv_path.append('A')
        elif label == 'B':
            inv_path.append('B')
    
    return inv_path[::-1]
            


T = np.zeros((G.number_of_nodes(), G.number_of_nodes()))

for i, j in tqdm(itertools.product(range(G.number_of_nodes()), repeat=2)):
    total_path = inverse_path(G.nodes[i]['path']) + G.nodes[j]['path']
    elem = np.eye(2, dtype=int)
    for label in total_path:
        elem = normalize_projective(generator_set[label] @ elem)

    # NOTE: overflow problem if we consider larger groups etc

    for key, elems in adjacent_elems.items():
        for elem_ in elems:
            if np.array_equal(elem, elem_):
                T[i, j] = prob_weights[key-1]



    

0it [00:00, ?it/s]

28224it [00:02, 10559.46it/s]
